## Purpose of our Program: 
The purpose of our program is to connect readers so they can get recommendations, provide reviews, and find new books to read.

## Purpose of my feature:
The purpose of my feature, the wishlist, is for readers to be able to keep track of the books that they want to read in the future.


## API Methods:

### 1. GET Method: Fetching Data
The API provides two `GET` routes:

#### Fetching All Books
```python
@wishlist_api.route('/books', methods=['GET'])
def get_books():
    """Retrieve all books from the database to display in a dropdown menu."""
    books = Book.query.all()
    books_list = [{'id': book.id, 'title': book.title, 'author': book.author} for book in books]
    return jsonify(books_list)
```
This function retrieves all books from the database and returns a JSON list of book objects.

#### Fetching Books in Wishlist
```python
@wishlist_api.route('/', methods=['GET'])
def get_wishlist():
    """Retrieve all books in the wishlist."""
    wishlist_items = Wishlist.query.all()
    books_in_wishlist = []
    for item in wishlist_items:
        book = Book.query.get(item.book_id)
        if book:
            books_in_wishlist.append({'id': book.id, 'title': book.title, 'author': book.author})
    return jsonify(books_in_wishlist)
```
This function retrieves books currently in the wishlist by cross-referencing the `Wishlist` model with the `Book` model.

### 2. POST Method: Adding Data
```python
@wishlist_api.route('/', methods=['POST'])
def add_book_to_wishlist():
    """Add a book to the wishlist."""
    if request.is_json:
        data = request.get_json()
        book_id = data.get('book_id')

        if not book_id:
            return jsonify({"error": "Missing book_id"}), 400

        book = Book.query.get(book_id)
        if not book:
            return jsonify({"error": "Book not found"}), 404

        existing_entry = Wishlist.query.filter_by(book_id=book_id).first()
        if existing_entry:
            return jsonify({"message": "Book already in wishlist"}), 200

        new_entry = Wishlist(book_id=book_id)
        db.session.add(new_entry)
        db.session.commit()

        return jsonify({"message": "Book added to wishlist"}), 201

    return jsonify({"error": "Request must be JSON"}), 415
```
This function:
- Extracts `book_id` from the JSON request body.
- Validates if the book exists in the database.
- Checks if the book is already in the wishlist.
- Adds the book to the wishlist and commits the transaction.
- Returns appropriate responses for missing or invalid data.

### 3. DELETE Method: Removing Data
```python
@wishlist_api.route('/<int:book_id>', methods=['DELETE'])
def delete_book_from_wishlist(book_id):
    """Delete a book from the wishlist."""
    wishlist_item = Wishlist.query.filter_by(book_id=book_id).first()

    if not wishlist_item:
        return jsonify({"error": "Book not found in wishlist"}), 404

    try:
        db.session.delete(wishlist_item)
        db.session.commit()
        return jsonify({"message": "Book removed from wishlist"}), 200
    except Exception as e:
        db.session.rollback()
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500
```
This function:
- Checks if the book exists in the wishlist.
- Deletes the book entry from the wishlist.
- Handles database commit errors properly.

## Handling Requests with Algorithms
The API demonstrates three fundamental control structures:

### 1. **Sequencing**
Each function follows a structured sequence:
1. Receive the request.
2. Validate the input.
3. Query the database.
4. Process and return the response.

### 2. **Selection (Conditional Statements)**
Conditional logic is used in:
- `POST` request: Checking if `book_id` is missing, if the book exists, and if it's already in the wishlist.
- `DELETE` request: Ensuring the book is in the wishlist before attempting deletion.

### 3. **Iteration (Looping)**
Iteration is applied in the `GET` wishlist method:
```python
for item in wishlist_items:
    book = Book.query.get(item.book_id)
    if book:
        books_in_wishlist.append({'id': book.id, 'title': book.title, 'author': book.author})
```
This loop retrieves book details for each wishlist item.

## API Call Example
### Making a Request Using `fetch`
```javascript
fetch('/api/wishlist/', {
    method: 'POST',
    headers: {
        'Content-Type': 'application/json'
    },
    body: JSON.stringify({ book_id: 1 })
})
.then(response => response.json())
.then(data => console.log(data));
```
### Handling the Response
```javascript
fetch('/api/wishlist/')
    .then(response => response.json())
    .then(data => console.log(data))
    .catch(error => console.error('Error:', error));
```

## Error Handling and Response Variations
| Condition | Response |
|-----------|----------|
| `book_id` is missing | `{ "error": "Missing book_id" }` (400) |
| Book does not exist | `{ "error": "Book not found" }` (404) |
| Book already in wishlist | `{ "message": "Book already in wishlist" }` (200) |
| Successfully added | `{ "message": "Book added to wishlist" }` (201) |
| Successfully deleted | `{ "message": "Book removed from wishlist" }` (200) |
| Book not in wishlist | `{ "error": "Book not found in wishlist" }` (404) |

This document comprehensively details the API's logic, request handling, and error responses.



## Testing API's on Postman

Return List of Books:
![img](../images/get_books.png)

Return Wishlist
![img](../images/get_wishlist.png)

Add a Book to the Wishlist
![img](../images/add_wishlist.png)

Delete a Book From the Wishlist
![img](../images/delete_wishlist.png)






### Lists
Lists are used to handle rows of data, such as when restoring the Wishlist table from a list of dictionaries. Here, data is a list of dictionaries, each representing a row to be added to the database.
The restore method iterates over the list and adds each dictionary as a row in the database.

In [ ]:
data = [{"book_id": 1}, {"book_id": 2}, {"book_id": 3}]
Wishlist.restore(data)

### Dictionaries
Each dictionary contains column names as keys (id, book_id) and their respective values.
This is particularly useful when serializing database objects for use in APIs, such as returning JSON responses.

In [ ]:
def read(self):
    return {
        "id": self.id,
        "book_id": self.book_id,
    }

### CRUD
The function get_wishlist() retrieves all rows from the wishlist table as a list of Wishlist objects. The SQLAlchemy ORM is the third-party library used here.

In [ ]:
def get_wishlist():
    return Wishlist.query.all()

The returned list can be further processed, e.g., converted into a JSON response. This converts the list of Wishlist objects into a list of dictionaries. 


In [ ]:
wishlist_items = [item.read() for item in get_wishlist()]


The Wishlist model provides methods for handling individual columns (id, book_id):

Create: Using add_to_wishlist(book_id):


In [ ]:
item = Wishlist(book_id=book_id)
db.session.add(item)
db.session.commit()

Read: Using read():


In [ ]:
wishlist_dict = item.read()


Delete: Using delete_from_wishlist(book_id):


In [ ]:
item = Wishlist.query.filter_by(book_id=book_id).first()
db.session.delete(item)
db.session.commit()

## Sequencing, Selection, and Iteration

The add_book_to_wishlist method demonstrates sequencing, selection, and iteration:






In [ ]:
# Route to add a book to the wishlist
@wishlist_api.route('/', methods=['POST'])
def add_book_to_wishlist():
    """Add a book to the wishlist."""
    if request.is_json:
        data = request.get_json()
        book_id = data.get('book_id')

        # Validate that book_id is provided
        if not book_id:
            return jsonify({"error": "Missing book_id"}), 400

        # Check if the book exists in the books database
        book = Book.query.get(book_id)
        if not book:
            return jsonify({"error": "Book not found"}), 404

        # Check if the book is already in the wishlist
        existing_entry = Wishlist.query.filter_by(book_id=book_id).first()
        if existing_entry:
            return jsonify({"message": "Book already in wishlist"}), 200

        # Add the book to the wishlist
        new_entry = Wishlist(book_id=book_id)
        db.session.add(new_entry)
        db.session.commit()

        return jsonify({"message": "Book added to wishlist"}), 201

    return jsonify({"error": "Request must be JSON"}), 415


Sequencing: Sequentially processes the request, validates the input, checks database records, and adds a new entry.

Selection: Uses if conditions to handle cases like missing book_id, non-existent books, or duplicate entries.

Iteration: Iterates through existing records using a loop to build the wishlist response.

Parameters and Return Types

Parameters:

The POST request body must be JSON and include `book_id`.

Example body:

{
  "book_id": 1
}

Return Type: JSON responses are created using `jsonify`, returning:

Success messages (`201` or `200` status).

Error messages (`400`, `404`, or `415` status).

## Making a Request


In [ ]:
// Adding a book to the wishlist
fetch('/api/wishlist/', {
    method: 'POST',
    headers: {
        'Content-Type': 'application/json'
    },
    body: JSON.stringify({ book_id: 1 })
})
.then(response => response.json())
.then(data => console.log(data));